# Introduction

In [1]:
from ipywidgets import interact
from upsetjs_jupyter_widget import UpSetWidget
import pandas as pd

In [2]:
w = UpSetWidget[str]()
# w.theme = 'dark'
w.mode = 'hover'

In [3]:
w.from_dict(dict(one = ['a', 'b', 'c', 'e', 'g', 'h', 'k', 'l', 'm'], two = ['a', 'b', 'd', 'e', 'j'], three = ['a', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm']))
w

UpSetWidget(value=None, elems=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm'], height_ratios…

In [4]:
df = pd.DataFrame(dict(
    one=[1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1], 
    two=[1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0], 
    three=[1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1]
), index=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm'])
w.from_dataframe(df)
w

UpSetWidget(value=None, elems=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm'], height_ratios…

In [21]:
w.sets

[UpSetSet(name=one, elems={'b', 'c', 'l', 'k', 'h', 'a', 'g', 'm', 'e'}),
 UpSetSet(name=three, elems={'i', 'l', 'e', 'h', 'a', 'g', 'j', 'm', 'f'}),
 UpSetSet(name=two, elems={'a', 'b', 'j', 'd', 'e'})]

In [22]:
w.combinations

[UpSetSetIntersection(name=one, sets={'one'}, elems={'b', 'c', 'l', 'k', 'h', 'a', 'g', 'm', 'e'}),
 UpSetSetIntersection(name=three, sets={'three'}, elems={'i', 'l', 'e', 'h', 'a', 'g', 'j', 'm', 'f'}),
 UpSetSetIntersection(name=(one ∩ three), sets={'one', 'three'}, elems={'h', 'a', 'l', 'm', 'g', 'e'}),
 UpSetSetIntersection(name=two, sets={'two'}, elems={'a', 'b', 'j', 'd', 'e'}),
 UpSetSetIntersection(name=(one ∩ two), sets={'one', 'two'}, elems={'a', 'e', 'b'}),
 UpSetSetIntersection(name=(three ∩ two), sets={'two', 'three'}, elems={'j', 'a', 'e'})]

In [23]:
w

UpSetWidget(value=None, elems=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm'], height_ratios…

In [24]:
def selection_changed(s, ws):
    # w.selection = ws["name"] if ws else None
    return s, ws # s["name"] if s else None
interact(selection_changed, s=10, ws=w)

interactive(children=(IntSlider(value=10, description='s', max=30, min=-10), UpSetWidget(value=None, elems=['a…

<function __main__.selection_changed(s, ws)>